# Travel Agent Planner with RagaAI Catalyst Integration

This Jupyter notebook demonstrates the integration of Catalyst, a powerful tracing and monitoring tool, with a Travel Agent Planner system. AgentNeo provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.


# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [ ]:
from ragaai_catalyst.tracers import Tracer
from ragaai_catalyst import RagaAICatalyst
import os
import requests
from dotenv import load_dotenv
import openai
from openai import OpenAI


INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [2]:
# Load environment variables
load_dotenv()

# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Initialize providers
catalyst = RagaAICatalyst(
    access_key="*****",
    secret_key="*****",
    base_url="https://catalyst.raga.ai/api",
)

Token(s) set successfully


In [4]:
tracer = Tracer(
    project_name="tracer",
    dataset_name="testing",
    tracer_type="anything",
    metadata={
        "model": "gpt-4o-mini",
        "environment": "production"
    },
    pipeline={
        "llm_model": "gpt-4o-mini",
        "vector_store": "faiss",
        "embed_model": "text-embedding-ada-002",
    }
)

In [5]:
tracer.start()

# Travel Agent Tools
Now, let's define our Travel Agent Planner Tools with Catalyst integration.

In [6]:
@tracer.trace_llm(name="llm_call")
def llm_call(prompt, max_tokens=512, model="gpt-4o-mini"):
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()

# Tools outside agents
@tracer.trace_tool(name="weather_tool")
def weather_tool(destination):

    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    params = {"q": destination, "appid": api_key, "units": "metric"}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]

        return f"{weather_description.capitalize()}, {temperature:.1f}°C"
    except requests.RequestException:
        return "Weather data not available."


@tracer.trace_tool(name="currency_converter_tool")
def currency_converter_tool(amount, from_currency, to_currency):
    api_key = os.environ.get("EXCHANGERATE_API_KEY")
    base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"

    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()

        if data["result"] == "success":
            rate = data["conversion_rate"]
            return amount * rate
        else:
            return None
    except requests.RequestException:
        return None


@tracer.trace_tool(name="flight_price_estimator_tool")
def flight_price_estimator_tool(origin, destination):
    # This is a mock function. In a real scenario, you'd integrate with a flight API.
    api_key = os.environ.get("FLIGHT_API_KEY")
    # Implement actual API call here
    return f"Estimated price from {origin} to {destination}: $500-$1000"


# Agent with persona
@tracer.trace_agent(name="itinerary_agent")
class ItineraryAgent:
    def __init__(self, persona="Itinerary Agent"):
        self.persona = persona

    def plan_itinerary(self, user_preferences, duration=3):
        itinerary_prompt = f"""
You are a travel expert named {self.persona}.
Based on the following user preferences, create a {duration}-day travel itinerary.

User Preferences:
{user_preferences}

Itinerary:
"""
        return llm_call(itinerary_prompt, max_tokens=512)



# Running the Analysis
Now let's create a  main function for our Travel Agent Planner which recall all tools 

In [7]:
# Main function

@tracer.trace_agent(name="travel_agent")
def travel_agent():
    print("Welcome to the Personalized Travel Planner!\n")

    # Get user input
    # user_input = input("Please describe your ideal vacation: ")
    user_input = "karela, 10 days, $100, nature"

    # Extract preferences
    preferences_prompt = f"""
Extract key travel preferences from the following user input:
"{user_input}"

Please provide the extracted information in this format:
Destination:
Activities:
Budget:
Duration (in days):
"""
    extracted_preferences = llm_call(preferences_prompt)
    print("\nExtracted Preferences:")
    print(extracted_preferences)

    # Parse extracted preferences
    preferences = {}
    for line in extracted_preferences.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            preferences[key.strip()] = value.strip()

    # Validate extracted preferences
    required_keys = ["Destination", "Activities", "Budget", "Duration (in days)"]
    if not all(key in preferences for key in required_keys):
        print("\nCould not extract all required preferences. Please try again.")
        return

    # Fetch additional information
    weather = weather_tool(preferences["Destination"])
    print(f"\nWeather in {preferences['Destination']}: {weather}")

    # origin = input("Please enter your departure city: ")
    origin = "delhi"
    flight_price = flight_price_estimator_tool(origin, preferences["Destination"])
    print(flight_price)

    # Plan itinerary
    itinerary_agent = ItineraryAgent()
    itinerary = itinerary_agent.plan_itinerary(
        extracted_preferences, int(preferences["Duration (in days)"])
    )
    print("\nPlanned Itinerary:")
    print(itinerary)

    # Currency conversion
    budget_amount = float(preferences["Budget"].replace("$", "").replace(",", ""))
    converted_budget = currency_converter_tool(budget_amount, "USD", "INR")
    if converted_budget:
        print(f"\nBudget in INR: {converted_budget:.2f} INR")
    else:
        print("\nCurrency conversion not available.")

    # Generate travel summary
    summary_prompt = f"""
Summarize the following travel plan:

Destination: {preferences['Destination']}
Activities: {preferences['Activities']}
Budget: {preferences['Budget']}
Duration: {preferences['Duration (in days)']} days
Itinerary: {itinerary}
Weather: {weather}
Flight Price: {flight_price}

Travel Summary:
"""
    travel_summary = llm_call(summary_prompt, max_tokens=2048)
    print("\nTravel Summary:")
    print(travel_summary)

In [8]:
# Main function to run the travel agent
def main():
    travel_agent()

# Ensure the script runs only when executed directly
if __name__ == "__main__":
    main()
    tracer.stop()


Welcome to the Personalized Travel Planner!



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Extracted Preferences:
Destination: Karela  
Activities: Nature  
Budget: $100  
Duration (in days): 10

Weather in Karela: Weather data not available.
Estimated price from delhi to Karela: $500-$1000


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Planned Itinerary:
**10-Day Itinerary for Kerala: Nature-Focused Adventure on a Budget of $100**

**Day 1: Arrival in Kochi**
- Arrive in Kochi.
- Take a walk along Fort Kochi Beach and enjoy the sunset.
- Dinner at a local eatery (budget-friendly options available).
- Overnight stay in a budget hostel or guesthouse in Fort Kochi.

**Day 2: Explore Kochi**
- Visit the famous Chinese Fishing Nets at sunrise.
- Explore Mattancherry Palace and Jew Town for a glimpse of the local culture.
- Lunch at a local café.
- Visit the Kerala Folklore Museum in the afternoon (entrance fee is minimal).
- Dinner at a local restaurant.
- Overnight stay in Fort Kochi.

**Day 3: Munnar Travel**
- Travel to Munnar by local bus (approx. $2).
- Check into a budget homestay or hostel.
- Visit the Tea Museum (entry fee is low).
- Explore the nearby tea plantations.
- Enjoy a simple dinner at your accommodation.
- Overnight stay in Munnar.

**Day 4: Nature Walks in Munnar**
- Take a guided nature walk through 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Travel Summary:
**Travel Summary for Kerala Nature-Focused Adventure**

- **Destination:** Kerala
- **Activities:** Nature exploration
- **Budget:** $100
- **Duration:** 10 days

**Itinerary Overview:**

- **Day 1:** Arrive in Kochi, walk along Fort Kochi Beach, and enjoy a sunset. Stay in a budget hostel.
- **Day 2:** Explore Kochi’s Chinese Fishing Nets, Mattancherry Palace, and Kerala Folklore Museum. Overnight in Fort Kochi.
- **Day 3:** Travel to Munnar via local bus, visit the Tea Museum, and enjoy local tea plantations. Stay in a budget homestay.
- **Day 4:** Go on a guided nature walk and visit Eravikulam National Park. Overnight in Munnar.
- **Day 5:** Hike Anamudi Peak, enjoying the natural beauty. Return to Munnar for an overnight stay.
- **Day 6:** Travel to Thekkady by local bus, explore Periyar Wildlife Sanctuary, and take a sunset boat ride. Overnight in Thekkady.
- **Day 7:** Engage in nature walks, visit a spice plantation, and enjoy local cuisine.

**Notes:** 
- Acti